In [1]:
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mc
import colorsys
import time 

Color Intensity setting

In [2]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])
    

Setting up functions to find distance and unit vectors

In [3]:
def distance(p1, p2):
    """
    Gives us the distance between two points in space
    
    Args:
        p1 : point 1 in space, NumPy array
        p2 : point 2 in space, NumPy array
    """
    return np.linalg.norm(p2-p1)

def normalise(vector):
    """
    Gives you the normal vector for a given vector
    
    Args:
        vector : The vector that needs to be normalised
    """
    return vector/(np.linalg.norm(vector)) 

def velocity(p1, p2, t1, t2):
    """
    Calculates the velocity between two points in space over time
    
    Args:
        p1 : Initial position, NumPy array
        p2 : Final position, NumPy array
        t1 : Initial time
        t2 : Final time
    """
    return (p2 - p1) / (t2 - t1)

def acceleration(v1, v2, t1, t2):
    """
    Calculates the acceleration between two velocities over time
    
    Args:
        v1 : Initial velocity, NumPy array
        v2 : Final velocity, NumPy array
        t1 : Initial time
        t2 : Final time
    """
    return (v2 - v1) / (t2 - t1)

Setting up the FABRIK Function

In [4]:
num_link = 4
reps=300000

def positions(phi, l=2*np.ones((num_link,))):
    """
    Find the position of each joint of the mechanism
    Args:
        phi : The angles of the joints with respect to the positive x axis
        l : Length of the links
    """
    x, y = np.zeros((num_link+1, 1)), np.zeros((num_link+1, 1))
    
    for i in range(1, num_link+1):
        x[i, :], y[i, :] = x[i-1, :] + l[i-1] * np.cos(phi[i-1]), y[i-1, :] + l[i-1] * np.sin(phi[i-1])
    
    return np.concatenate((x, y), axis=1)

p_dash = np.zeros((reps, num_link+1, 2))
v_dash = np.zeros((reps-1, num_link+1, 2))
a_dash = np.zeros((reps-2, num_link+1, 2))

for i in range(reps):
    alpha = np.random.uniform(-np.pi, np.pi, (num_link,))
    p_dash[i, :, :] = positions(phi=alpha)
    if i > 0:
        v_dash[i-1, :, :] = velocity(p_dash[i-1, :, :], p_dash[i, :, :], i-1, i)
    if i > 1:
        a_dash[i-2, :, :] = acceleration(v_dash[i-2, :, :], v_dash[i-1, :, :], i-2, i-1)
# print(p_dash[:, 4, :].shape)
# print(v_dash[:, 4, :].shape)
# print(a_dash[:, 4, :].shape)

In [5]:
def FABRIK_solver(aim, p, num_links=4, length=2*np.ones((4,)), start=np.array([[0,0]]), max_iterations=10, epsilon=1e-3):
    """
    Uses the FABRIK technique to solve for a planar manipulator, for multiple different initial configurations.
    
    Args:
        start : The position of the starting point, 1x2 matrix that is broadcasted to repx2 shape.
        aim : The desired position of the end effector 
        num_links : Number of links  
        length : Length of links
        max_iterations : The maximum number of iterations of the algorithm before deciding on outcome, default is 100
        epsilon : The maximum error that can be allowed between the estimated end effector 
                  position and the actual end effector position, default is 1e-3
    """
    reps = p.shape[0]
    thetas = np.zeros((reps, 1, num_links))
    v_dash = np.zeros_like(p[:-1])
    a_dash = np.zeros_like(p[:-2])
    
    if np.sum(length) < distance(aim, start):
        print("This point cannot be reached")
        return thetas, p, v_dash, a_dash

    for iter in range(max_iterations):  
        error = np.zeros((reps,))
        for rep in range(reps):
            error[rep] = distance(aim, p[rep, num_links, :])
            
        if (error >= epsilon).all():    
            target = np.repeat(aim, reps, axis=0)
            fixed = np.repeat(start, reps, axis=0)
            
            phi = np.zeros((reps,))
            vect = np.zeros((reps, 2))
            unit_vect = np.zeros((reps, 2))
            
            for link in range(num_links, 0, -1):
                for r in range(reps):
                    vect[r, :] = p[r, link-1, :] - target[r, :]
                    unit_vect[r, :] = normalise(vect[r, :])
                    phi[r] = np.arctan2(unit_vect[r, 1], unit_vect[r, 0])
                    
                p[:, link, :] = target
                p[:, link-1, :] = target + length[link-1] * np.transpose((np.array([np.cos(phi[:]), np.sin(phi[:])])))
                target = p[:, link-1, :]
                thetas[:, :, link-1] = np.reshape(np.arctan2(p[:, link, 1] - p[:, link-1, 1], p[:, link, 0] - p[:, link-1, 0]).reshape((reps, 1)), (reps, 1))

            alpha = np.zeros((reps,))
            vecto = np.zeros((reps, 2))
            norm_vecto = np.zeros((reps, 2))

            for link in range(num_links):
                for r in range(reps):
                    vecto[r, :] = p[r, link+1, :] - fixed[r, :]
                    norm_vecto[r, :] = normalise(vecto[r, :])
                    alpha[r] = np.arctan2(norm_vecto[r, 1], norm_vecto[r, 0])
                p[:, link, :] = fixed
                p[:, link+1, :] = fixed + length[link] * np.transpose((np.array([np.cos(alpha[:]), np.sin(alpha[:])])))
                fixed = p[:, link+1, :]
        
                thetas[:, :, link] = np.reshape(np.arctan2(p[:, link+1, 1] - p[:, link, 1], p[:, link+1, 0] - p[:, link, 0]), (reps, 1))
                
    if iter > 0:
        v_dash = p[1:] - p[:-1]
    if iter > 1:
        a_dash = v_dash[1:] - v_dash[:-1]

    return thetas, p, v_dash, a_dash

In [6]:
def fabrik(aim, p=p_dash[0,:,:], length=np.array([2,2,2,2]), start=np.array([0,0]), max_iterations=10, epsilon=1e-3):
    """
    Uses the FABRIK technique to solve for a planar manipulator, when solving one by one for each initial configuration
    
    Args:
        start : The position of the starting point
        aim : The desired position of the end effector 
        4 : Number of links  
        length : Length of links
        max_iterations : The maximum number of iterations of the algorithm before deciding on outcome, default is 100
        epsilon : The maximum error that can be allowed between the estimated end effector 
                  position and the actual end effector position, default is 1e-3
    """
    thetas = np.zeros((1, 4))
    velocities = np.zeros((max_iterations, 4, 2))
    accelerations = np.zeros((max_iterations-1, 4, 2))

    if np.sum(length) < distance(aim, start):
        print("This point cannot be reached")
        return thetas, p, velocities, accelerations

    for iter in range(max_iterations):
        if np.abs(aim - p[4, :]).all() >= epsilon:
            target = aim
            fixed = start

            for link in range(4, 0, -1):
                vect = p[link-1, :] - target
                unit_vect = normalise(vect)
                phi = np.arctan2(unit_vect[1], unit_vect[0])
                p[link, :] = target
                p[link-1, :] = target + length[link-1] * (np.array([np.cos(phi), np.sin(phi)]))
                target = p[link-1, :]
                thetas[:, link-1] = np.arctan2(p[link, 1] - p[link-1, 1], p[link, 0] - p[link-1, 0])

            for link in range(4):
                vecto = p[link+1, :] - fixed
                norm_vecto = normalise(vecto)
                alpha = np.arctan2(norm_vecto[1], norm_vecto[0])
                p[link, :] = fixed
                p[link+1, :] = fixed + length[link] * (np.array([np.cos(alpha), np.sin(alpha)]))
                fixed = p[link+1, :]
                thetas[:, link] = np.arctan2(p[link+1, 1] - p[link, 1], p[link+1, 0] - p[link, 0])

            if iter > 0:
                velocities[iter-1, :, :] = velocity(p_prev, p, iter-1, iter)
            if iter > 1:
                accelerations[iter-2, :, :] = acceleration(velocities[iter-2, :, :], velocities[iter-1, :, :], iter-2, iter-1)

            p_prev = np.copy(p)

    return thetas, p, velocities, accelerations

Testing the code

In [7]:
# final = np.array([[-1.5, -0.8]])

# start_time = time.time()

# thetas, points, v_dash, a_dash = FABRIK_solver(
#     aim=final, 
#     num_links=4, 
#     length=np.array([1, 1, 1, 1]),  # Ensure length is an array
#     start=np.array([[0, 0]]), 
#     max_iterations=100, 
#     epsilon=1e-3
# )

# end_time = time.time()
# time_taken = end_time - start_time

# print(f"Time taken: {time_taken} seconds")

# plt.figure(dpi=150)
# for i in range(0, reps):
#     plt.plot(points[i, :, 0], points[i, :, 1], "-", color=lighten_color("k", 0.5), linewidth="0.5", marker="o", label="Configuration")

# plt.plot(final[:, 0], final[:, 1], marker="o", color="k", label="Target End Effector Position")
# plt.plot(0, 0, marker="o", color="r", label="Start Point")

# plt.xlim(-4, 4)
# plt.ylim(-4, 4)
# plt.grid()
# plt.show()

# print(f"Thetas: {thetas}")
# print(f"Velocity Dash (v_dash): {v_dash}")
# print(f"Acceleration Dash (a_dash): {a_dash}")

# for i in range(len(points) - 1):
#     print(f"Distance between point {i} and {i+1}: {distance(points[i], points[i+1])}")
